Glorot初始化（使用逻辑激活函数）：

正态分布，其均值为0，方差为$\sigma^2=\frac{1}{fan_{avg}}$,或$-r$和$+r$之间的均匀分布，其中$r=\sqrt{\frac{3}{fan_{avg}}}$

其中$fan_{avg}=\frac{(fan_{in}+fan_{out})}{2}$

```python
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")
```

Variance Scaling初始化：

```python
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode='fan_avg',
                                                 distribution='uniform')
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)
```

ELU激活函数：$$
\mathrm{ELU}_\alpha(z) =
\begin{cases}
\alpha \left( \exp(z) - 1 \right) & \text{如果 } z < 0 \\
z & \text{如果 } z \geq 0
\end{cases}
$$

它的计算较慢但是收敛速度更快。

SELU激活函数： $\mathrm{SELU}(z)=\lambda \cdot \mathrm{ELU}(z)$

自归一化的条件：
1. 输入特征必须是标准化的（mean=0，std=1）
2. 每个隐藏层的权重使用LeCun正态初始化，即`kernel_initializer="lecun_normal"`
3. 网络架构必须是顺序的
4. 所有层都是密集层

通常SELU>ELU>leaky ReLU（及其变体）>ReLU>tanh>logistic

如果网络的架构不能自归一化，那么ELU的性能
可能会优于SELU（因为SELU在z=0时不平滑）。如果你非常关心运行时
延迟，那么你可能更喜欢leaky ReLU。如果你不想调整其他超参数，则
可以使用Keras使用的默认α值（例如，leaky ReLU为0.3）。如果你有
空闲时间和计算能力，则可以使用交叉验证来评估其他激活函数，例
如，如果网络过拟合，则为RReLU；如果你的训练集很大，则为PReLU。如果你将速度放在首位，那么
ReLU可能仍然是最佳选择。
也就是说，ReLU是迄今为止最常用的激活函数


leakyReLU:
```python
model = keras.models.Sequential([
    keras.layers.Dense(10, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(alpha=0.2),
    [...]
])
```

SELU:
```python
layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")
```

批量归一化算法：
1. $\mu_{\mathrm{B}} = \frac{1}{m_{\mathrm{B}}} \sum_{i=1}^{m_{\mathrm{B}}} x^{(i)}$

2. $\sigma_{\mathrm{B}}^2 = \frac{1}{m_{\mathrm{B}}} \sum_{i=1}^{m_{\mathrm{B}}} \left(x^{(i)} - \mu_{\mathrm{B}}\right)^2$

3. $\hat{x}^{(i)} = \frac{x^{(i)} - \mu_{\mathrm{B}}}{\sqrt{\sigma_{\mathrm{B}}^2 + \varepsilon}}$

4. $z^{(i)} = \gamma \otimes \hat{x}^{(i)} + \beta$

其中，$\mu_B$是输入均值的向量，在整个小批量B上评估;$\sigma_B$是输入标准差的向量，也在整个小批量中进行评估;$m_B$是小批量中的实例数量;$X_{(i)}$是实例$i$的零中心和归一化输入的向量;$\gamma$是该层的输出缩放参数向量;$\otimes$表示逐元素乘法;$\beta$是层的输出移动（偏移）参数向量，每个输入都通过其相应的移动参数进行偏移;$\epsilon$为平滑项;$z^{(i)}$是BN操作的输出，是输入的缩放和偏移版本。

在每个批归一化层中学习了四个参数向量：通过常规反向传播学习$\gamma$（输出缩放向量）和$\beta$（输出偏移向量），和使用指数移动平均值估
计的$\mu$（最终的输入均值向量）和$\sigma$（最终输入标准差向量）

In [2]:
#用keras实现批量归一化
from tensorflow import keras
from tensorflow.keras.layers import Dense
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Bat  (None, 784)               3136      
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_4 (Bat  (None, 300)               1200      
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_5 (Bat  (None, 100)              

In [3]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_3/gamma:0', True),
 ('batch_normalization_3/beta:0', True),
 ('batch_normalization_3/moving_mean:0', False),
 ('batch_normalization_3/moving_variance:0', False)]

In [5]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(100, kernel_initializer="he_normal", use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("elu"),
    keras.layers.Dense(10, activation="softmax")
])

Update $\hat{v}$:

$\hat{v}\leftarrow \hat{v}\times momentum + v\times(1-momentum)$

In [6]:
#梯度裁剪
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)